In [ ]:
rm -r tuner_dir

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [ ]:
! kaggle competitions download -c histopathologic-cancer-detection

In [ ]:
! mkdir data

In [ ]:
! unzip histopathologic-cancer-detection.zip -d data

**Problem Description:**

The challenge is a binary image classification task aimed at detecting metastatic cancer in small image patches extracted from larger digital pathology scans. Specifically, the goal is to develop an algorithm that can accurately classify these patches as either containing metastatic cancer (positive label) or not (negative label). The presence of cancer is determined solely by the presence of tumor tissue within the center 32x32 pixel region of each patch. This task has significant clinical relevance, as it directly addresses the critical issue of cancer metastasis detection.

**Data Description:**

The dataset is a modified version of the PatchCamelyon (PCam) benchmark dataset. It consists of a large number of small, color pathology images. Will Cukierski. Histopathologic Cancer Detection. https://kaggle.com/competitions/histopathologic-cancer-detection, 2018. Kaggle.

-   **Size:** The dataset comprises thousands of image patches, split into training and testing sets.
-   **Dimension:** Each image patch has a fixed dimension of 96x96 pixels with 3 color channels (RGB).
-   **Structure:**
    -   The data (which is all stored in `data/`) is organized into two subdirectories: `train/` and `test/`.
    -   The `train` folder contains images for training the model, and the `test` folder contains images for which predictions are to be made.
    -   A `train_labels.csv` file provides the ground truth labels for the training images, mapping image IDs to binary labels (0 or 1).
    -   The labels are only determined by the center 32x32 pixel region of each image.
    -   The outer area of the 96x96 images are provided to support fully convolutional network architectures.
-   **Format:** Images are in standard image formats (tif).
-   **Class Imbalance:** It's important to note that the dataset exhibits class imbalance, with an unequal distribution of positive and negative samples.
-   **No Duplicates:** The Kaggle version of the PCam dataset has been processed to remove duplicate images, ensuring a cleaner training set.

In [ ]:
!pip install numpy pandas matplotlib seaborn opencv-python albumentations tensorflow keras-tuner scikit-learn Pillow

In [ ]:
# --- Standard Libraries ---
import os
import io
import warnings
warnings.filterwarnings('ignore')  # Cleaner output

# --- Data Manipulation & Visualization ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- Image Processing & Augmentation ---
import cv2
import albumentations as A  # Efficient augmentations
from albumentations.pytorch import ToTensorV2  # Optional for conversion

# --- TensorFlow & Deep Learning ---
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses, metrics, callbacks
from tensorflow.keras.applications import ResNet50, EfficientNetB3  # Backbones
from tensorflow.keras.layers import (GlobalAveragePooling2D, Dense,
                                    Dropout, BatchNormalization)
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Legacy loader
from tensorflow.keras import backend as K

# --- Hyperparameter Tuning & Utilities ---
import keras_tuner as kt
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from PIL import Image

In [ ]:
print("TensorFlow Version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

In [ ]:
# Load labels and check dataset size
train_labels = pd.read_csv('data/train_labels.csv')
train_dir = 'data/train'
print(f"Total training samples: {len(train_labels)}")
print(f"Test directory file count: {len(os.listdir('data/test'))}")  # Verify test size

In [ ]:
# Display the first few rows of the labels
print(train_labels.head())

In [ ]:
# Class distribution analysis
plt.figure(figsize=(10, 6))
ax = sns.countplot(x='label', data=train_labels, palette='viridis')

# Add percentages to the plot
total = len(train_labels)
for p in ax.patches:
    percentage = f'{100 * p.get_height()/total:.1f}%'
    ax.annotate(percentage, (p.get_x() + p.get_width()/2, p.get_height()),
                ha='center', va='bottom', fontsize=12)

plt.title(f'Class Distribution (Total Samples: {total})', fontsize=14)
plt.xlabel('Class (0 = Negative, 1 = Positive)', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.show()

In [ ]:
# Numerical summary
class_counts = train_labels['label'].value_counts()
print(f"Class 0 (Negative): {class_counts[0]} samples ({class_counts[0]/total:.1%})")
print(f"Class 1 (Positive): {class_counts[1]} samples ({class_counts[1]/total:.1%})")

In [ ]:
# Get positive & negative samples
num_samples = 5
positive_samples = train_labels[train_labels['label'] == 1].sample(n=num_samples, random_state=42)
negative_samples = train_labels[train_labels['label'] == 0].sample(n=num_samples, random_state=42)

In [ ]:
# Function to plot samples with center region highlighted
def plot_class_samples(df, class_label, num_samples=5, title_suffix=""):
    samples = df[df['label'] == class_label].sample(n=num_samples, random_state=42)
    fig, axes = plt.subplots(1, num_samples, figsize=(20, 4))
    fig.suptitle(f"Class {class_label} Samples {title_suffix}", y=1.05, fontsize=16)

    for ax, (idx, row) in zip(axes, samples.iterrows()):
        img = cv2.imread(os.path.join(train_dir, row['id'] + '.tif'))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Highlight center region
        cv2.rectangle(img, (32, 32), (64, 64), (255, 0, 0), 2)  # Red rectangle

        ax.imshow(img)
        ax.axis('off')
    plt.show()

# Plot both classes with consistent formatting
plot_class_samples(train_labels, 1, title_suffix="(Positive - Cancer Present)")
plot_class_samples(train_labels, 0, title_suffix="(Negative - No Cancer)")

In [ ]:
def compare_class_color_distributions(num_samples=100):
    plt.figure(figsize=(15, 5))

    # Sample images from both classes
    pos_samples = train_labels[train_labels['label'] == 1].sample(num_samples)
    neg_samples = train_labels[train_labels['label'] == 0].sample(num_samples)

    # Load and process images
    def get_center_pixels(samples):
        pixels = []
        for _, row in samples.iterrows():
            img = cv2.imread(os.path.join(train_dir, row['id'] + '.tif'))
            center_region = img[32:64, 32:64]  # Focus on diagnostic area
            pixels.append(center_region.reshape(-1, 3))
        return np.vstack(pixels)

    pos_pixels = get_center_pixels(pos_samples)
    neg_pixels = get_center_pixels(neg_samples)

    # Plot distributions
    channels = ['Red', 'Green', 'Blue']
    for i in range(3):
        plt.subplot(1, 3, i+1)
        sns.kdeplot(pos_pixels[:, i], color='r', label='Positive', fill=True)
        sns.kdeplot(neg_pixels[:, i], color='b', label='Negative', fill=True)
        plt.title(f'{channels[i]} Channel Distribution')
        plt.xlabel('Pixel Intensity')
        plt.xlim(0, 255)
        if i == 0: plt.legend()

    plt.suptitle("Color Channel Distributions in Center Region (n=100 per class)", y=1.05)
    plt.tight_layout()
    plt.show()

compare_class_color_distributions()

In [ ]:
# Calculate class weights for loss function

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels['label']),
    y=train_labels['label'].values
)
class_weights = dict(enumerate(class_weights))
print(f"Class Weights: {class_weights}")

In [ ]:
def create_augmenter():
    return A.Compose([
        # Spatial transforms
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.ShiftScaleRotate(
            shift_limit=0.05,
            scale_limit=0.05,
            rotate_limit=15,
            p=0.5,
            border_mode=cv2.BORDER_CONSTANT
        ),

        # Color transforms
        A.RandomBrightnessContrast(p=0.3),
        A.CLAHE(p=0.3),
        A.HueSaturationValue(p=0.3),

        # Remove CropNonEmptyMaskIfExists
        A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0
        ),
        ToTensorV2()
    ])

In [ ]:
# Split data into train/validation
train_df, val_df = train_test_split(
    train_labels,
    test_size=0.2,
    stratify=train_labels['label'],  # Preserve class balance
    random_state=42
)
print(f"Train size: {len(train_df)}, Validation size: {len(val_df)}")

In [ ]:
# Verify stratified split
def check_split_distribution(train_df, val_df):
    train_ratio = train_df['label'].mean()
    val_ratio = val_df['label'].mean()
    orig_ratio = train_labels['label'].mean()

    print(f"Original positive ratio: {orig_ratio:.4f}")
    print(f"Train positive ratio:    {train_ratio:.4f}")
    print(f"Val positive ratio:      {val_ratio:.4f}")

    assert np.isclose(train_ratio, orig_ratio, atol=0.005), "Train split not stratified!"
    assert np.isclose(val_ratio, orig_ratio, atol=0.005), "Val split not stratified!"

check_split_distribution(train_df, val_df)

In [ ]:
def visualize_augmentations(df, n=3):
    samples = df.sample(n)
    fig, axes = plt.subplots(n, 3, figsize=(15, 4*n))
    aug = create_augmenter()

    for i, (_, row) in enumerate(samples.iterrows()):
        img = cv2.imread(os.path.join(train_dir, row['id'] + '.tif'))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Original (normalized for visualization)
        axes[i,0].imshow(img/255.0)
        axes[i,0].set_title(f"Original (Class {row['label']})")

        # Apply augmentation twice
        for j in [1, 2]:
            augmented = aug(image=img)['image']
            # Convert tensor to numpy and denormalize
            augmented = augmented.numpy().transpose(1, 2, 0)
            augmented = augmented * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]
            augmented = np.clip(augmented, 0, 1)

            axes[i,j].imshow(augmented)
            axes[i,j].set_title(f"Augmented {j}")

    plt.tight_layout()
    plt.show()

visualize_augmentations(train_labels)

### **Key Considerations for Architecture Design**

1.  **Focus on Center Region**:

    -   The critical diagnostic region is small (32x32 pixels), so architectures must prioritize local feature extraction over global context.

    -   Solutions: **Crop the center region** during preprocessing, use **attention mechanisms**, or design **weighted loss functions** to emphasize the center.

2.  **Class Imbalance**:

    -   Use **class weights**, **Focal Loss**, or **oversampling** to prevent bias toward the majority class.

3.  **Computational Efficiency**:

    -   96x96 images are smaller than typical ImageNet resolutions, but transfer learning with pretrained models (e.g., ResNet, EfficientNet) can still work with minor adaptations.

### **Architecture Comparison**

| **Model** | **Strengths** | **Weaknesses** | **Use Case** |
| --- | --- | --- | --- |
| **Simple CNN** | Fast, lightweight | Limited feature hierarchy | Baseline for quick testing |
| **ResNet-50 + Attention** | Strong pretrained features, dynamic focus | Computationally heavy | High-accuracy deployment |
| **EfficientNet-B3** | Efficient, scalable, good out-of-box performance | Requires tuning for medical images | Resource-constrained environments |

In [ ]:
# ---------------------------
# 1. Simple CNN with Center Crop
# ---------------------------
def build_simple_cnn():
    model = models.Sequential([
        layers.Input(shape=(96, 96, 3)),
        layers.Rescaling(1./255),
        layers.Normalization(
            mean=[0.485, 0.456, 0.406],
            variance=[0.229**2, 0.224**2, 0.225**2]
        ),
        layers.Cropping2D(cropping=((32, 32), (32, 32))),  # 96x96 → 32x32
        layers.Conv2D(32, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.GlobalAveragePooling2D(),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-3),
        loss=losses.BinaryCrossentropy(),
        metrics=[
            metrics.AUC(name='auc'),
            metrics.Precision(name='prec'),
            metrics.Recall(name='rec')
        ]
    )
    return model

# ---------------------------
# 2. ResNet-50 + Spatial Attention
# ---------------------------
def spatial_attention_block(input_tensor):
    """Focus attention on center region with learnable weights"""
    # Channel aggregation
    avg_pool = layers.GlobalAveragePooling2D(keepdims=True)(input_tensor)
    max_pool = layers.GlobalMaxPooling2D(keepdims=True)(input_tensor)
    concat = layers.Concatenate(axis=-1)([avg_pool, max_pool])

    # Spatial attention with center bias
    attention = layers.Conv2D(1, (3,3), padding='same', activation='sigmoid')(concat)

    # Add center region prior (32x32 at center)
    h, w = K.int_shape(input_tensor)[1], K.int_shape(input_tensor)[2]
    center_mask = np.zeros((1, h, w, 1))
    cy, cx = h//2, w//2
    center_mask[:, cy-16:cy+16, cx-16:cx+16, :] = 0.5  # Soft prior
    attention = layers.Add()([attention, tf.constant(center_mask, dtype=tf.float32)])

    return layers.Multiply()([input_tensor, attention])

def build_resnet_attention():
    inputs = layers.Input(shape=(96, 96, 3))
    x = layers.Rescaling(1./255)(inputs)
    x = layers.Normalization(
        mean=[0.485, 0.456, 0.406],
        variance=[0.229**2, 0.224**2, 0.225**2]
    )(x)

    base_model = ResNet50(
        include_top=False,
        input_tensor=x,
        pooling=None
    )
    x = base_model.output

    x = spatial_attention_block(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs, outputs)

    # Freezing strategy
    for layer in base_model.layers[:150]:
        layer.trainable = False
    for layer in base_model.layers[150:]:
        layer.trainable = True

    model.compile(
        optimizer=optimizers.Adam(1e-4),
        loss=losses.BinaryFocalCrossentropy(gamma=2),
        metrics=[metrics.AUC(name='auc')]
    )
    return model

# ---------------------------
# 3. EfficientNet-B3 + Context Preservation
# ---------------------------
def build_efficientnet():
    inputs = layers.Input(shape=(96, 96, 3))

    # Context-aware preprocessing
    x = layers.RandomCrop(64, 64)(inputs)  # Random context around center
    x = layers.Resizing(224, 224)(x)       # Upscale for EfficientNet

    x = layers.Rescaling(1./255)(x)
    x = layers.Normalization(
        mean=[0.485, 0.456, 0.406],
        variance=[0.229**2, 0.224**2, 0.225**2]
    )(x)

    base_model = EfficientNetB3(
        include_top=False,
        weights='imagenet',
        input_tensor=x
    )
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs, outputs)

    # Partial fine-tuning
    base_model.trainable = True
    for layer in base_model.layers[:-20]:
        layer.trainable = False

    model.compile(
        optimizer=optimizers.Adam(1e-3),
        loss=losses.BinaryCrossentropy(label_smoothing=0.1),
        metrics=[metrics.AUC(name='auc')]
    )
    return model

### **Key Features Summary Table**

| Model | Input Size | Output Size | Parameters | Key Features |
| --- | --- | --- | --- | --- |
| **Simple CNN** | 96x96x3 | 32x32x3 | ~100K | Center cropping, lightweight |
| **ResNet-50+Attention** | 96x96x3 | 7x7x2048 | ~23M | Spatial attention with center prior, focal loss |
| **EfficientNet-B3** | 224x224x3 | 7x7x1536 | ~12M | Context preservation via random crops, label smoothing |

In [ ]:
def create_dataset(df, augment=False):
    """Create optimized TF Dataset for TIFF images"""
    image_paths = df['id'].apply(lambda x: f"data/train/{x}.tif").tolist()
    labels = df['label'].values.astype(np.float32)

    # Create dataset from slices
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    def parse_image(path, label):
        """Read and decode TIFF image"""
        img_bytes = tf.io.read_file(path)
        try:
            # Use PIL to decode TIFF
            image = tf.py_function(
                lambda x: np.array(Image.open(io.BytesIO(x.numpy())).convert('RGB')),
                [img_bytes],
                tf.uint8
            )
            image.set_shape([96, 96, 3])  # Enforce expected shape
            return image, label
        except:
            print(f"Error reading {path}, skipping")
            return tf.zeros([96, 96, 3], dtype=tf.uint8), label

    def preprocess(image, label):
        """Standard preprocessing"""
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = tf.image.per_image_standardization(image)
        return image, label

    # Map with proper argument handling
    dataset = dataset.map(
        lambda p, l: tf.py_function(parse_image, [p, l], [tf.uint8, tf.float32]),
        num_parallel_calls=tf.data.AUTOTUNE
    )
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

    if augment:
        def augment_fn(image, label):
            """Augmentation pipeline"""
            image = tf.image.random_flip_left_right(image)
            image = tf.image.random_brightness(image, 0.1)
            image = tf.image.random_contrast(image, 0.9, 1.1)
            return image, label
        dataset = dataset.map(augment_fn, num_parallel_calls=tf.data.AUTOTUNE)

    return dataset.batch(64).prefetch(2)

# Create datasets
train_ds = create_dataset(train_df, augment=True)
val_ds = create_dataset(val_df)

In [ ]:
# Check first batch
sample_images, sample_labels = next(iter(train_ds))
print(f"Image batch shape: {sample_images.shape}")  # Should be (64, 96, 96, 3)
print(f"Label batch shape: {sample_labels.shape}")  # Should be (64,)

# Visualize sample
plt.imshow(sample_images[0].numpy().astype('float32'))
plt.show()

In [ ]:
# Select a sample of IDs (here, the first 10)
sample_ids = train_df['id'].head(10).tolist()

# Construct full file paths and check if they exist
for img_id in sample_ids:
    path = f"data/train/{img_id}.tif"
    exists = tf.io.gfile.exists(path)
    print(f"File path: {path} exists: {exists}")

In [ ]:
# Choose a sample file path
sample_path = "data/train/d6b4fe580210f5ea61b7c109b5c2e5cc734c3010.tif"

# Read the file contents
img_bytes = tf.io.read_file(sample_path)

# Convert tensor to bytes (using .numpy() because we're in eager mode)
img_bytes = img_bytes.numpy()

try:
    # Attempt to load the image with PIL
    img = Image.open(io.BytesIO(img_bytes))
    # Convert to RGB (in case it's in a different mode)
    img = img.convert("RGB")
    # Convert to a NumPy array
    img_arr = np.array(img, dtype=np.uint8)
    print("Image loaded successfully with shape:", img_arr.shape)
except Exception as e:
    print("Error loading image:", e)

In [ ]:
for images, labels in train_ds.take(1):
    print("Batch image shape:", images.shape)
    print("Batch labels shape:", labels.shape)


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
from PIL import Image, UnidentifiedImageError

from tensorflow.keras import layers, models, optimizers, losses, metrics
from tensorflow.keras.applications import ResNet50, EfficientNetB3
from tensorflow.keras import backend as K
import keras_tuner

# ===============================
# 1. Model Building Functions
# ===============================

# ---------------------------
# Simple CNN with Center Crop
# ---------------------------
def build_simple_cnn():
    model = models.Sequential([
        layers.Input(shape=(96, 96, 3)),
        layers.Rescaling(1./255),
        layers.Normalization(
            mean=[0.485, 0.456, 0.406],
            variance=[0.229**2, 0.224**2, 0.225**2]
        ),
        layers.Cropping2D(cropping=((32, 32), (32, 32))),  # 96x96 → 32x32
        layers.Conv2D(32, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.GlobalAveragePooling2D(),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-3),
        loss=losses.BinaryCrossentropy(),
        metrics=[
            metrics.AUC(name='auc'),
            metrics.Precision(name='prec'),
            metrics.Recall(name='rec')
        ]
    )
    return model

# ---------------------------
# ResNet-50 + Spatial Attention
# ---------------------------
def spatial_attention_block(input_tensor):
    """Focus attention on center region with learnable weights."""
    # Aggregate channels using average and max pooling
    avg_pool = layers.GlobalAveragePooling2D(keepdims=True)(input_tensor)
    max_pool = layers.GlobalMaxPooling2D(keepdims=True)(input_tensor)
    concat = layers.Concatenate(axis=-1)([avg_pool, max_pool])

    # Use (1,1) convolution on the pooled features to generate an attention map
    attention = layers.Conv2D(1, (1,1), padding='same', activation='sigmoid')(concat)

    # Create a center prior: for a feature map of shape (h, w), highlight the center cell.
    h, w = K.int_shape(input_tensor)[1], K.int_shape(input_tensor)[2]
    center_mask = np.zeros((1, h, w, 1), dtype=np.float32)
    if h is not None and w is not None:
        cy, cx = h // 2, w // 2
        center_mask[:, cy, cx, :] = 0.5  # Soft prior at the center cell
    attention = layers.Add()([attention, tf.constant(center_mask)])

    return layers.Multiply()([input_tensor, attention])

def build_resnet_attention():
    inputs = layers.Input(shape=(96, 96, 3))
    x = layers.Rescaling(1./255)(inputs)
    x = layers.Normalization(
        mean=[0.485, 0.456, 0.406],
        variance=[0.229**2, 0.224**2, 0.225**2]
    )(x)

    # Use ResNet50 backbone without pretrained weights (or use 'imagenet' if input size is compatible)
    base_model = ResNet50(
        include_top=False,
        weights=None,
        input_tensor=x,
        pooling=None
    )
    x = base_model.output

    x = spatial_attention_block(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs, outputs)

    # Freeze early layers for efficiency.
    for layer in base_model.layers[:150]:
        layer.trainable = False
    for layer in base_model.layers[150:]:
        layer.trainable = True

    model.compile(
        optimizer=optimizers.Adam(1e-4),
        loss=losses.BinaryFocalCrossentropy(gamma=2),
        metrics=[metrics.AUC(name='auc')]
    )
    return model

# ---------------------------
# EfficientNet-B3 + Context Preservation
# ---------------------------
def build_efficientnet():
    inputs = layers.Input(shape=(96, 96, 3))

    # Context-aware preprocessing: random crop and resizing.
    x = layers.RandomCrop(64, 64)(inputs)  # Crop random 64x64 patch.
    x = layers.Resizing(224, 224)(x)        # Resize to 224x224 for EfficientNet.

    x = layers.Rescaling(1./255)(x)
    x = layers.Normalization(
        mean=[0.485, 0.456, 0.406],
        variance=[0.229**2, 0.224**2, 0.225**2]
    )(x)

    base_model = EfficientNetB3(
        include_top=False,
        weights='imagenet',
        input_tensor=x
    )
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs, outputs)

    # Partial fine-tuning: freeze most layers except the last 20.
    base_model.trainable = True
    for layer in base_model.layers[:-20]:
        layer.trainable = False

    model.compile(
        optimizer=optimizers.Adam(1e-3),
        loss=losses.BinaryCrossentropy(label_smoothing=0.1),
        metrics=[metrics.AUC(name='auc')]
    )
    return model

# ===============================
# 2. Data Pipeline
# ===============================
def create_dataset(df, augment=False):
    """Create optimized TF Dataset for TIFF images with enforced shape."""
    image_paths = df['id'].apply(lambda x: f"data/train/{x}.tif").tolist()
    labels = df['label'].values.astype(np.float32)

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    def parse_image(path, label):
        """Read and decode TIFF image, enforcing known shape and catching load errors."""
        img_bytes = tf.io.read_file(path)

        def load_img(x):
            try:
                # Load image with PIL and convert to RGB.
                img = Image.open(io.BytesIO(x))
                img = img.convert('RGB')
                return np.array(img, dtype=np.uint8)
            except (UnidentifiedImageError, Exception) as e:
                print(f"Error loading image {path.numpy() if hasattr(path, 'numpy') else path}: {e}")
                # Return a black image if loading fails.
                return np.zeros((96, 96, 3), dtype=np.uint8)

        image = tf.py_function(load_img, [img_bytes], tf.uint8)
        image = tf.reshape(image, [96, 96, 3])  # Explicitly set shape
        return image, label

    def preprocess(image, label):
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = tf.image.per_image_standardization(image)
        return image, label

    dataset = dataset.map(parse_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

    if augment:
        def augment_fn(image, label):
            image = tf.image.random_flip_left_right(image)
            image = tf.image.random_brightness(image, 0.1)
            image = tf.image.random_contrast(image, 0.9, 1.1)
            return image, label
        dataset = dataset.map(augment_fn, num_parallel_calls=tf.data.AUTOTUNE)

    return dataset.batch(64).prefetch(tf.data.AUTOTUNE)

# (Assumes train_df and val_df are defined DataFrames.)
train_ds = create_dataset(train_df, augment=True)
val_ds = create_dataset(val_df)

# ===============================
# 3. Hyperparameter Tuning Across Architectures
# ===============================
def build_hypermodel(hp):
    # Let the tuner choose among the three architectures.
    model_type = hp.Choice("model_type", ["simple_cnn", "resnet_attention", "efficientnet"])

    if model_type == "simple_cnn":
        model = build_simple_cnn()
        lr = hp.Float("lr", 1e-4, 1e-2, sampling="LOG", default=1e-3)
        model.compile(
            optimizer=optimizers.Adam(learning_rate=lr),
            loss=losses.BinaryCrossentropy(),
            metrics=[metrics.AUC(name="auc")]
        )
    elif model_type == "resnet_attention":
        model = build_resnet_attention()
        lr = hp.Float("lr", 1e-5, 1e-3, sampling="LOG", default=1e-4)
        model.compile(
            optimizer=optimizers.Adam(learning_rate=lr),
            loss=losses.BinaryFocalCrossentropy(gamma=2),
            metrics=[metrics.AUC(name="auc")]
        )
    else:  # efficientnet
        model = build_efficientnet()
        lr = hp.Float("lr", 1e-4, 1e-2, sampling="LOG", default=1e-3)
        model.compile(
            optimizer=optimizers.Adam(learning_rate=lr),
            loss=losses.BinaryCrossentropy(label_smoothing=0.1),
            metrics=[metrics.AUC(name="auc")]
        )
    return model

tuner = keras_tuner.Hyperband(
    build_hypermodel,
    objective="val_auc",
    max_epochs=10,
    factor=3,
    directory="tuner_dir",
    project_name="architecture_comparison"
)

# Run the hyperparameter search.
tuner.search(train_ds, validation_data=val_ds, epochs=10, verbose=3)

# Summarize results.
tuner.results_summary()

# ===============================
# 4. Analyzing Tuning Results
# ===============================
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters:", best_hp.values)

best_model = tuner.get_best_models(num_models=1)[0]
eval_results = best_model.evaluate(val_ds)
print("Evaluation results on validation dataset:", eval_results)

# Create a summary DataFrame of top trials.
trials = tuner.oracle.get_best_trials(num_trials=10)
results = []
for trial in trials:
    hp_values = trial.hyperparameters.values
    val_auc = trial.score
    results.append({**hp_values, "val_auc": val_auc})
results_df = pd.DataFrame(results).sort_values("val_auc", ascending=False)
print(results_df)

# Plot validation AUC by model architecture.
plt.figure(figsize=(8,6))
plt.bar(results_df["model_type"], results_df["val_auc"])
plt.xlabel("Model Architecture")
plt.ylabel("Validation AUC")
plt.title("Validation AUC by Model Architecture")
plt.show()
